# FASHION-MNIST WITH PYTORCH

In [0]:
!pip install torch
!pip install torchvision
!pip install matplotlib
!wget "https://github.com/udacity/deep-learning-v2-pytorch/blob/master/intro-to-pytorch/helper.py"

--2018-12-06 16:51:53--  https://github.com/udacity/deep-learning-v2-pytorch/blob/master/intro-to-pytorch/helper.py
Resolving github.com (github.com)... 192.30.253.113, 192.30.253.112
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘helper.py.1’

helper.py.1             [ <=>                ]  81.29K  --.-KB/s    in 0.03s   

2018-12-06 16:51:53 (3.08 MB/s) - ‘helper.py.1’ saved [83246]



In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import torch
from torch import nn
from torch import optim
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
import helpers #this was uploaded from my local drive
import torch.nn.functional as F

ModuleNotFoundError: ignored

In [0]:
#lets download the dataset 
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# download and load the mnist dataset
trainset = datasets.FashionMNIST('FashionMNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

In [0]:
#let view the dataset

dataiter = iter(trainloader)
image, label = dataiter.next()
print(type(image))
print(image.shape)
print(label.shape)

In [0]:
#lets view the image
plt.imshow(image[1].numpy().squeeze(), cmap='Greys_r')

# Now, Lets build the network model

METHOD 1: 

In [0]:
model = nn.Sequential(nn.Linear(784, 128),
                     nn.ReLU(),
                     nn.Linear(128, 64),
                     nn.ReLU(),
                     nn.Linear(64, 10),
                     nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss() #negative log-likelihood loss
optimizer = optim.SGD(model.parameters(), lr = 0.03) #where lr is the learning rate
epochs = 10

for e in range(epochs):
  running_loss = 0
  for images, labels in trainloader:
    
    #flatten images
    images = images.view(images.shape[0], -1) # or images.view(image.shape[0], 784)
    
    optimizer.zero_grad()
    
    # forward pass, get our logits
    log_probability = model(images) #or output = model.forward(images)

    
    loss = criterion(log_probability, labels)
    
    loss.backward()
    
    optimizer.step()
    
    running_loss += loss.item()
  else:
    print(f"Training Loss: {running_loss/len(trainloader)}")


In [0]:
#Lets predict

images, labels = next(iter(trainloader))

img = images[0].view(1, 784)

#lets turn off the gradients

with torch.no_grad():
  logits = model.forward(img)

#Output of the network are logits, need to take softmax of probabilities

ps = F.softmax(logits, dim=1)

#ps = nn.LogSoftmax(logits)

helpers.view_classify(img.view(1, 28, 28), ps, version='Fashion')

METHOD 2:

In [0]:
#raw method
class Network(nn.Module):
  def __init__(self):
    super().__init__()
    self.FC1 = nn.Linear(784, 256)
    self.FC2 = nn.Linear(256, 128)
    self.FC3 = nn.Linear(128, 64)
    self.OUT = nn.Linear(64, 10)
#     self.relu = nn.ReLU()
#     self.softmax = nn.LogSoftmax(dim=1)
    
#   def forward(self, x):
#     #i could flatten the images before they enter the network\
#     x = x.view(x.shape[0], -1)
    
#     x = self.FC1(x)
#     x = self.relu = nn.ReLU(x)
#     x = self.FC2(x)
#     x = self.relu = nn.ReLU(x)
#     x = self.FC3(x)
#     x = self.relu = nn.ReLU(x)
#     x = self.OUT(x)
#     x = self.softmax(x)
#     return x
  
  ## OR
  
  def forward(self, x):
        #i could flatten the images before they enter the network\
    x = x.view(x.shape[0], -1)
    
    x = F.relu(self.FC1(x))
    x = F.relu(self.FC2(x))
    x = F.relu(self.FC3(x))
    x = F.log_softmax(self.OUT(x), dim=1)
    return x

In [0]:
model = Network()
model

In [0]:
#Now, to train the network
criterion = nn.NLLLoss() #negative log-likelihood loss
optimizer = optim.Adam(model.parameters(), lr = 0.003) #where lr is the learning rate
# optimizer = optim.SGD(model.parameters(), lr = 0.003) #where lr is the learning rate
epochs = 10

In [0]:
for e in range(epochs):
  running_loss = 0
  for images, labels in trainloader:
    
    optimizer.zero_grad()
    
    # forward pass, get our logits
    log_probability = model(images) #or output = model.forward(images)

    
    loss = criterion(log_probability, labels)
    
    loss.backward()
    
    optimizer.step()
    
    running_loss += loss.item()
  else:
    print(f"Training Loss: {running_loss/len(trainloader)}")

In [0]:
#Lets predict

images, labels = next(iter(trainloader))

img = images[0].view(1, 784)

#lets turn off the gradients

with torch.no_grad():
  logits = model.forward(img)

#Output of the network are logits, need to take softmax of probabilities

ps = F.softmax(logits, dim=1)

#ps = nn.LogSoftmax(logits)

helpers.view_classify(img.view(1, 28, 28), ps, version='Fashion')

# INFERENCE AND VALIDATION

IN ORDER FOR US TO VALIDATE THE MODEL WE HAVE TO PASS DATA THAT IS NOT IN THE TRAIN SET (TEST SET). IN ORDER TO KNOW HOW WELL THE MODEL IS DOING

In [0]:
#lets download the dataset 
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# download and load the mnist dataset
trainset = datasets.FashionMNIST('FashionMNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)


testset = datasets.FashionMNIST('FashionMNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

In [0]:
# now, lets define the model
from torch import nn, optim
import torch.nn.functional as F

class Network(nn.Module):
  def __init__(self):
    super().__init__()
    self.FC1 = nn.Linear(784, 256)
    self.FC2 = nn.Linear(256, 128)
    self.FC3 = nn.Linear(128, 64)
    self.FC4 = nn.Linear(64, 10)
  
  def forward(self, x):
    #flatten the images 
    x = x.view(x.shape[0], -1)
    
    x = F.relu(self.FC1(x))
    x = F.relu(self.FC2(x))
    x = F.relu(self.FC3(x))
    x = F.log_softmax(self.FC4(x), dim=1)
    
    return x

In [0]:
# now, lets pass one batch of the testset through the model 

model = Network()

images, labels = next(iter(testloader))

#now, lets get the class probability

pa = torch.exp(model(images))
print(pa.shape)

In [0]:
top_p, top_class = pa.topk(1, dim=1)

#looks at the most likely class for the first 10 examples

print(top_class[:10,:])

In [0]:
#now, lets compare the models prediction with the actual values of the classes

equals = top_class == labels.view(*top_class.shape)
equals

In [0]:
# so we can get the accuracy by getting the mean of equals (since we have jut zeros and ones as the output)
accuracy = torch.mean(equals.type(torch.FloatTensor))
print(f'Accuracy: {accuracy.item()*100}%')

In [0]:
# so, lets train the network and test the accuracy
criterion = NLLLoss()
optimiser = optim.Adam(model.parameter(), lr=0.003)

epochs = 30
steps = 0

trainlosses, testlosses = [], []
for e in range(epochs):
  running_loss = 0
  for images, labels in trainloader:
    
    optimizer.zero_grad()
    
    # forward pass, get our logits
    log_probability = model(images) #or output = model.forward(images)

    
    loss = criterion(log_probability, labels)
    
    loss.backward()
    
    optimiser.step()
    
    running_loss += loss.item()
  else:
    # we will define the testinfg process here
    print(f"Training Loss: {running_loss/len(trainloader)}")

In [0]:
database = { }

def check_username_exist(username, password):
  if username not database:
    print("That user does not exist")
    return "a"
  elif password not password:
    print("Password incorrect")
    return "b"
  else:
    return "c"
      

def add_to_database(username, password):
  database[username] = password
  
def registration(username, password, confirm_password):
  if(password != confirm_password):
    return print("does not match")
  else:
    add_to_database(username, password)
    
def add_user_process():
  try:
    name = input("Please choose a username: ")
    pwd = input("Please choose a password: ")
    confirmPwd = input("Re-enter your password: ")
    check = pwd == confirmPwd
    while check == False:
      print("password is match")
      add_user_process()
    registration(name, pwd, confirmPwd)
    print("Registration Successfull!!")
  except Exception e:
    print (e)

In [0]:


loop = False

while loop == True:
  username = input("please input your username: ")
  password = input("please input your password: ")
  value = check_username_exist(username, password)
  
  if value == "a":
    print("we have no such user. Please register on our database")
    response = input("INPUT 1 TO CREATE NEW USER \n INPUT 2 TO RETRY LOGIN: ")
      if response == "1":
        add_user_process()
        print("\n\n\n\n\n=====Now you can try to login in again=====")
        return True
      if response == "2":
        return True
      
  return True

  elif value == "b":
    response = input("INPUT 1 TO CREATE NEW USER \n INPUT 2 TO RETRY LOGIN: ")
    if response == "1":
      add_user_process()
      print("\n\n\n\n\n=====Now you can try to login in again=====")
      return True
    if response == "2":
      return True
  return True

  elif value == "c":
    print("an unknown error occured")
    return True
  
  else:
    print("Login Successfull. \n\n Welcome!!!")
    return False
      